In [19]:
def loadDataSet(fileName):
    f = open(fileName,'r')
    dataSet=[]
    current = 1
    subSet = []
    for line in f.readlines():
        line=line.strip('\n')
        tmpList = line.split(' ')
        index = tmpList[0]
        item = tmpList[1]

        if int(index) != current:
            dataSet.append(subSet)
            subSet = []
            subSet.append(item)
            current += 1
        else:
            subSet.append(item)
    dataSet.append(subSet)
    return dataSet    

def createC1(dataSet):
    C1 = []
    for data in dataSet:
        for item in data:
            if not [item] in C1:
                C1.append([item])
    return [frozenset(x) for x in C1]

def createLk(dataSet,Ck,minSupport):
    countCk = {}
    for data in dataSet:
        for subSet in Ck:
            if subSet.issubset(data):
                if not subSet in countCk:
                    countCk[subSet] = 1
                else:
                    countCk[subSet] += 1
    totalNum = len(dataSet)
    Lk = []
    countLk = {}
    for key in countCk:
        support = countCk[key]/totalNum
        if support >= minSupport:
            Lk.append(key)
            countLk[key] = support
            
    return Lk,countLk

def createCk(Lk,k):
    ### input
    # type of Lk : frozenset in list  
    # e.g. [frozenset({'A'}), frozenset({'C'}), frozenset({'B'}), frozenset({'E'})]
    # type of k  : int
    ###########
    ### output
    # type of Ck : frozenset in list
    Ck = []
    Lk_len = len(Lk)
    for i in range(Lk_len):
        for j in range(i+1,Lk_len):
            tmpSet = Lk[i] | Lk[j]
            if len(tmpSet) == k and not tmpSet in Ck:
                count = 0
                for item in Lk:
                    if set(item).issubset(tmpSet):
                        count += 1
                        #print(item,tmpSet,count)
                if count == k:
                    Ck.append(tmpSet)
    
    return Ck

def apriori(dataSet,minSupport =0.5):
    L = []
    C1 = createC1(dataSet)
    Lk,countL1 = createLk(dataSet,C1,minSupport)
    L.append(Lk)
    
    k = 2
    while len(L[k-2]) > 0: 
        Ck = createCk(Lk,k)
        Lk,countLk = createLk(dataSet,Ck,minSupport)
        L.append(Lk)
        k += 1
    return L[k-3]

def generateRules(dataSet,L,minConfidence=0.6):
    ruleList = []
    for i in range(len(L)):
        
    

SyntaxError: unexpected EOF while parsing (<ipython-input-19-aecf8c2717b2>, line 91)

In [16]:
dataSet = loadDataSet('data.txt')
L = Apriori(dataSet,0.5)
print(L)

[frozenset({'A', 'C', 'D'}), frozenset({'B', 'C', 'E'})]


In [113]:

frozenset(['a','b'])

frozenset({'a', 'b'})

In [4]:
a = [1,2,3]
b = [3,4]
c = frozenset(a) | set(b)
len(c)

4

In [18]:
frozenset(['a','b']) - frozenset(['a'])

frozenset({'b'})